<a href="https://colab.research.google.com/github/RG2806/ai_spotle/blob/master/CNN-82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/RG2806/ai_spotle.git

Cloning into 'ai_spotle'...
remote: Enumerating objects: 4057, done.
remote: Counting objects: 100% (4057/4057), done.
remote: Compressing objects: 100% (4054/4054), done.
remote: Total 4057 (delta 16), reused 4030 (delta 1), pack-reused 0
Receiving objects: 100% (4057/4057), 34.64 MiB | 23.38 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Sequential #Initialise our neural network model as a sequential network
from keras.layers import Conv2D #Convolution operation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation#Applies activation function
from keras.layers import Dropout#Prevents overfitting by randomly converting few outputs to zero
from keras.layers import MaxPooling2D # Maxpooling function
from keras.layers import Flatten # Converting 2D arrays into a 1D linear vector
from keras.layers import Dense # Regular fully connected neural network
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

import pandas as pd
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, GlobalAveragePooling2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras.backend as k
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [3]:
def load_data(dataset_path):
  classes = ['Fear','Sad','Happy']  #We will be dealing with seven different types of emotions.
  data = []
  labels =[]
  with open(dataset_path, 'r') as file:
      for line_no, line in enumerate(file.readlines()):
          if 0 < line_no <= 35887:
            line = line.split(',')
            image_data = np.asarray([int(x) for x in line[1:]]).reshape(48,48)#Creating a list out of the string then converting it into a 2-Dimensional numpy array.
            image_data =image_data.astype(np.float32)/255.0
            data.append(image_data)
            labels.append(classes.index(line[0]))
            data.append(cv2.flip(image_data, 1))
            labels.append(classes.index(line[0]))
  data = np.expand_dims(data, -1)   
  labels = to_categorical(labels, num_classes = 3)
    
  return np.array(data), np.array(labels)

In [4]:
dataset_path = "/content/ai_spotle/aithon2020_level2_traning.csv" 
train_data, train_labels = load_data(dataset_path)
#train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = test_size,random_state = seed)

print("Number of images in Training set:", len(train_data))

Number of images in Training set: 21634


In [5]:
train_data.shape

(21634, 48, 48, 1)

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=101,shuffle=True)
#y_train = np_utils.to_categorical(y_train, 3)
#y_test = np_utils.to_categorical(y_test, 3)

In [ ]:
num_features = 64
width, height = 48, 48
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))
model.summary()

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint('/content/input/model.h5', monitor='val_accuracy', verbose=1, save_best_only=True,mode='max')
checkpointer2= ModelCheckpoint('/content/input2/model2.h5', monitor='val_loss', verbose=1, save_best_only=True,mode='min')
callbacks = [lr_reducer, checkpointer,checkpointer2]

In [ ]:
# With Augmentation
bs = 64
epochs = 100

aug = ImageDataGenerator(rotation_range=20, 
                         width_shift_range=0.1, 
                         height_shift_range=0.1,  
                         fill_mode="nearest")

H = model.fit_generator(aug.flow(x_train, y_train, batch_size=bs),
                        validation_data=(x_test, y_test), 
                        steps_per_epoch=len(x_train)//bs,
                        callbacks=callbacks,
                        shuffle=True,
                        epochs=epochs)

In [10]:
scores = model.evaluate(x_test, y_test, batch_size=bs)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

68/68 [==============================] - 1s 16ms/step - loss: 0.4844 - accuracy: 0.8202
Loss: 0.4843789041042328
Accuracy: 0.8201987743377686


In [11]:
import sklearn.metrics as metrics
y_pred=model.predict(x_test)
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(metrics.classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.71      0.63      0.67      1051
           1       0.73      0.81      0.77      1326
           2       0.94      0.93      0.93      1950

    accuracy                           0.82      4327
   macro avg       0.80      0.79      0.79      4327
weighted avg       0.82      0.82      0.82      4327



In [12]:
model.save_weights("/content/model.h5")
print("Saved model to disk")

Saved model to disk


In [15]:
import os
from tensorflow import keras
def Data():
    # images are 48x48
    # N = 35887
    X=[]
    Y=[]
    for count, filename in enumerate(os.listdir("/content/ai_spotle/test/0")):
          img = image.load_img("/content/ai_spotle/test/0/"+filename, grayscale=True)
          #show_img=image.load_img('/content/hawon.jpg', grayscale=False, target_size=(200, 200))
          x = image.img_to_array(img)
          x = np.array(x , dtype = np.float32)/255.0
          X.append(x)
          Y.append(0)
    for count, filename in enumerate(os.listdir("/content/ai_spotle/test/1")):
          img = image.load_img("/content/ai_spotle/test/1/"+filename, grayscale=True)
          #show_img=image.load_img('/content/hawon.jpg', grayscale=False, target_size=(200, 200))
          x = image.img_to_array(img)
          x = np.array(x , dtype = np.float32)/255.0
          X.append(x)
          Y.append(1)
    for count, filename in enumerate(os.listdir("/content/ai_spotle/test/2")):
          img = image.load_img("/content/ai_spotle/test/2/"+filename, grayscale=True)
          #show_img=image.load_img('/content/hawon.jpg', grayscale=False, target_size=(200, 200))
          x = image.img_to_array(img)
          x = np.array(x , dtype = np.float32)/255.0
          X.append(x)
          Y.append(2)
    return X,Y

X,Y=Data()
X = np.expand_dims(X, -1)   
Y = to_categorical(Y, num_classes = 3)
import sklearn.metrics as metrics
y_pred=model.predict(X)
matrix = metrics.confusion_matrix(Y.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)
print(metrics.classification_report(Y.argmax(axis=1), y_pred.argmax(axis=1)))

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


[[ 649  319   56]
 [ 187  964   96]
 [  68   75 1631]]
              precision    recall  f1-score   support

           0       0.72      0.63      0.67      1024
           1       0.71      0.77      0.74      1247
           2       0.91      0.92      0.92      1774

    accuracy                           0.80      4045
   macro avg       0.78      0.78      0.78      4045
weighted avg       0.80      0.80      0.80      4045

